<a href="https://colab.research.google.com/github/ShinwooChoi/ESAA-OB/blob/main/9_15_ESAA_OB_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

범위: 핸즈온 7장 앙상블 학습과 랜덤 포레스트 p.257-272

##7.5 부스팅

정의: 여러 개 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

에이다부스트/그레이디언트 부스팅이 인기

###7.5.1 에이다부스트

- 이전 예측기를 보완하는 새로운 예측기를 만드는 방법으로, 과소적합했던 훈련 샘플의 가중치를 더 높여 새로운 예측기가 어려운 샘플을 점점 더 잘 학습하도록 만듦.

작동 방식

1. 첫 번째 분류기를 학습시켜 예측 수행

2. 잘못 분류된 샘플의 가중치를 높임

3. 두 번째 분류기는 업데이트된 가중치를 사용하여 학습

4. 이 과정을 반복하며 새로운 예측기를 계속 추가

특징

- 잘못 분류된 샘플의 가중치는 반복마다 절반 정도 증가

- 연속된 학습 방식으로 경사 하강법과 유사 (비용 함수 최소화)

- 모든 예측기 학습 후 앙상블은 배깅, 페이스팅과 비슷하지만 각 예측기마다 다른 가중치가 적용됨

단점

- 이전 예측기 결과에 의존하기 때문에 병렬화나 분할 불가능

- 배깅이나 페이스팅보다 확장성이 낮음

가중치 계산

- 예측기 가중치 αj = η log((1-rj)/rj)
(정확할수록 커지고, 무작위 추측이면 0, 더 나쁘면 음수가 됨)

- 샘플 가중치 업데이트: 잘못 분류된 샘플의 가중치 증가

최종 예측

- 모든 예측기의 예측을 합산하고 가중치를 더해 가장 큰 값을 갖는 클래스를 선택

사이킷런 구현

- AdaBoostClassifier, AdaBoostRegressor 제공

- 다중 클래스에서는 SAMME, SAMME.R 알고리즘 사용

- 기본 추정기는 깊이 1의 결정 트리

- 예시: 200개의 얕은 결정 트리 사용, learning_rate=0.5

TIP

- 과대적합이 발생하면 추정기 수를 줄이거나 규제를 강화


###7.5.2 그레이디언트 부스팅

-에이다부스트처럼 앙상블에 예측기를 순차적으로 추가

- 차이는 샘플 가중치 수정 대신 이전 예측기의 잔여 오차에 새로운 예측기 학습

- 결정 트리 사용 시 그레이디언트 트리 부스팅(GBRT)라고 부름

- GradientBoostingRegressor로 구현 가능

핵심 개념

- learning_rate: 각 트리의 기여도 조절, 작을수록 많은 트리 필요하지만 성능 ↑

- shrinkage(축소): 작은 학습률로 규제 강화

- 조기 종료: staged_predict()로 검증 오차 확인 후 최적 트리 수 결정

- warm_start=True: 기존 트리에 이어서 학습, 오차 향상 없으면 멈춤

- subsample: 학습 샘플 비율 조절, 무작위 선택으로 분산 ↓, 속도 ↑ (확률적 그레이디언트 부스팅)

XGBoost

- 최적화된 GBRT 구현, 속도·확장성·이식성 우수

- 사이킷런과 유사한 API 제공

- 자동 조기 종료 등 다양한 기능 지원



##7.6 스태킹

스태킹은 여러 예측기의 예측을 단순 합이 아닌 메타 학습기로 학습해 최종 예측 생성

각 예측기의 출력을 새로운 입력으로 사용, 블렌더(메타 학습기)가 최종 결과 만듦

일반적으로 홀드 아웃 세트를 사용해 블렌더 학습

훈련 세트를 서브셋으로 나눠 첫 번째 레이어 학습 → 홀드 아웃으로 예측 생성 → 블렌더 학습

여러 레이어로 확장 가능, 각 레이어가 차례로 예측 생성

사이킷런은 직접 지원하지 않으나 직접 구현하거나 DESlib 같은 라이브러리 사용 가능

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import package
import numpy as np
import os

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y=make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

y_pred = sum(tree.predict(X_new) for tree in [tree_reg1, tree_reg2, tree_reg3])


In [ ]:
#y=pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val=train_test_split(X, y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val, y_pred)
        for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=np.int64(114))

In [ ]:
gbrt=GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error=float("inf")
error_going_up=0
for n_estimators in range(1,120):
    gbrt.n_estimators=n_estimators
    gbrt.fit(X_train, y_train)
    y_pred=gbrt.predict(X_val)
    val_error=mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error=val_error
        error_going_up=0
    else:
        error_going_up+=1
        if error_going_up==5:
            break #조기 종료

In [ ]:
import xgboost

xgb_reg=xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred=xgb_reg.predict(X_val)

In [ ]:
xgb_reg=xgboost.XGBRegressor(early_stopping_rounds=2)
xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)])
y_pred=xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.36307
[1]	validation_0-rmse:0.26243
[2]	validation_0-rmse:0.18987
[3]	validation_0-rmse:0.14526
[4]	validation_0-rmse:0.11085
[5]	validation_0-rmse:0.08880
[6]	validation_0-rmse:0.06951
[7]	validation_0-rmse:0.06090
[8]	validation_0-rmse:0.05628
[9]	validation_0-rmse:0.05393
[10]	validation_0-rmse:0.05341
[11]	validation_0-rmse:0.05332
[12]	validation_0-rmse:0.05407
